Dataset Loading

In [123]:
import pandas as pd 
import numpy as np  
df = pd.read_csv("Housing.csv")
df = df.drop('furnishingstatus', axis=1, errors='ignore')

df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,13300000,7420.0,NaN,2,3,yes,no,no,no,yes,2,yes
1,12250000,8960.0,4.0,4,4,yes,no,no,no,yes,3,no
2,12250000,NaN,3.0,2,2,yes,no,yes,no,no,2,yes
3,12215000,NaN,4.0,2,2,yes,no,yes,no,yes,3,yes
4,11410000,7420.0,4.0,1,2,yes,yes,yes,no,yes,2,no
...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000.0,2.0,1,1,yes,no,yes,no,no,2,no
541,1767150,2400.0,3.0,1,1,no,no,no,no,no,0,no
542,1750000,3620.0,2.0,1,1,yes,no,no,no,no,0,no
543,1750000,2910.0,3.0,1,1,no,no,no,no,no,0,no


In [124]:

df.info

<bound method DataFrame.info of         price    area  bedrooms  bathrooms  stories mainroad guestroom  \
0    13300000  7420.0       NaN          2        3      yes        no   
1    12250000  8960.0       4.0          4        4      yes        no   
2    12250000     NaN       3.0          2        2      yes        no   
3    12215000     NaN       4.0          2        2      yes        no   
4    11410000  7420.0       4.0          1        2      yes       yes   
..        ...     ...       ...        ...      ...      ...       ...   
540   1820000  3000.0       2.0          1        1      yes        no   
541   1767150  2400.0       3.0          1        1       no        no   
542   1750000  3620.0       2.0          1        1      yes        no   
543   1750000  2910.0       3.0          1        1       no        no   
544   1750000  3850.0       3.0          1        2      yes        no   

    basement hotwaterheating airconditioning  parking prefarea  
0         no  

In [125]:
df.isnull().sum()

price               0
area               15
bedrooms            9
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
dtype: int64

In [126]:
df.mainroad.unique()

array(['yes', 'no'], dtype=object)

Train-Test Split

In [127]:
from sklearn.model_selection import train_test_split
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train.head()
 

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,7420.0,NaN,2,3,yes,no,no,no,yes,2,yes
1,8960.0,4.0,4,4,yes,no,no,no,yes,3,no
2,NaN,3.0,2,2,yes,no,yes,no,no,2,yes
3,NaN,4.0,2,2,yes,no,yes,no,yes,3,yes
4,7420.0,4.0,1,2,yes,yes,yes,no,yes,2,no


SimpleImputer Transformer on Area

In [128]:
from sklearn.impute import SimpleImputer
#define Imputer
si = SimpleImputer(missing_values=np.nan, strategy='mean')
#train it
si.fit(X_train[['area']])

#transform will return a numpy array that will be assigned to area column of dataframe
X_train['area'] = si.transform(X_train[['area']])
X_test['area'] = si.transform(X_test[['area']])
X_train.head()




,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,7420.000000,NaN,2,3,yes,no,no,no,yes,2,yes
1,8960.000000,4.0,4,4,yes,no,no,no,yes,3,no
2,5419.631829,3.0,2,2,yes,no,yes,no,no,2,yes
3,5419.631829,4.0,2,2,yes,no,yes,no,yes,3,yes
4,7420.000000,4.0,1,2,yes,yes,yes,no,yes,2,no


Simple Imputer transformer on Number of bed rooms
Note: In case of number of bedrooms, the best way I think is to train a model based on area and bedroom numbers but it will make current model more complex so i'll use simple imputer with median strategy on it

In [129]:
#train 2nd imputer for bedrooms
si1 = SimpleImputer(missing_values=np.nan, strategy='median')
#train imputer
si1.fit(X_train[['bedrooms']])

#transform the columns

X_train[['bedrooms']] = si1.transform(X_train[['bedrooms']])
X_test[['bedrooms']] = si1.transform(X_test[['bedrooms']])


X_train.head()


,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,7420.000000,3.0,2,3,yes,no,no,no,yes,2,yes
1,8960.000000,4.0,4,4,yes,no,no,no,yes,3,no
2,5419.631829,3.0,2,2,yes,no,yes,no,no,2,yes
3,5419.631829,4.0,2,2,yes,no,yes,no,yes,3,yes
4,7420.000000,4.0,1,2,yes,yes,yes,no,yes,2,no


Applying One hot encoding on Mainroad column first

In [130]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Ensure that the 'mainroad' column is of type string or number
X_train['mainroad'] = X_train['mainroad'].astype(str)
X_test['mainroad'] = X_test['mainroad'].astype(str)

# Initialize the OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32)

# Fit the encoder on the 'mainroad' column in the training set
ohe.fit(X_train[['mainroad']])

# Transform the 'mainroad' column in both train and test sets
X_train[['mainroad']] = ohe.transform(X_train[['mainroad']])
X_test[['mainroad']] = ohe.transform(X_test[['mainroad']])

X_train.tail()

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
431,3180.0,4.0,1,2,1,no,yes,no,yes,0,no
432,6060.0,3.0,1,1,1,yes,yes,no,no,0,no
433,3480.0,4.0,1,2,0,no,no,no,no,1,no
434,3792.0,4.0,1,2,1,no,no,no,no,0,no
435,4040.0,2.0,1,1,1,no,no,no,no,0,no


Applying One Hot Encoding on all other columns with binary outcomes

In [131]:
# Ensure that the 'guestroom' column is of type string or number
X_train['guestroom'] = X_train['guestroom'].astype(str)
X_test['guestroom'] = X_test['guestroom'].astype(str)

# Initialize the OneHotEncoder
ohe1 = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32)

# Fit the encoder on the 'guestroom' column in the training set
ohe1.fit(X_train[['guestroom']])

# Transform the 'mainroad' column in both train and test sets
X_train[['guestroom']] = ohe1.transform(X_train[['guestroom']])
X_test[['guestroom']] = ohe1.transform(X_test[['guestroom']])



# Initialize the OneHotEncoder
ohe2 = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32)

# Fit the encoder on the 'basement' column in the training set
ohe2.fit(X_train[['basement']])

# Transform the 'basement' column in both train and test sets
X_train[['basement']] = ohe2.transform(X_train[['basement']])
X_test[['basement']] = ohe2.transform(X_test[['basement']])



# Initialize the OneHotEncoder
ohe3 = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32)

# Fit the encoder on the 'hotwaterheating' column in the training set
ohe3.fit(X_train[['hotwaterheating']])

# Transform the 'hotwater heating' column in both train and test sets
X_train[['hotwaterheating']] = ohe3.transform(X_train[['hotwaterheating']])
X_test[['hotwaterheating']] = ohe3.transform(X_test[['hotwaterheating']])




# Initialize the OneHotEncoder
ohe4 = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32)

# Fit the encoder on the 'airconditioning' column in the training set
ohe4.fit(X_train[['airconditioning']])

# Transform the 'airconditioning' column in both train and test sets
X_train[['airconditioning']] = ohe4.transform(X_train[['airconditioning']])
X_test[['airconditioning']] = ohe4.transform(X_test[['airconditioning']])


# Initialize the OneHotEncoder
ohe5 = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32)

# Fit the encoder on the 'prefarea' column in the training set
ohe5.fit(X_train[['prefarea']])

# Transform the 'prefarea' column in both train and test sets
X_train[['prefarea']] = ohe5.transform(X_train[['prefarea']])
X_test[['prefarea']] = ohe5.transform(X_test[['prefarea']])






X_train.head()




,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,7420.000000,3.0,2,3,1,0,0,0,1,2,1
1,8960.000000,4.0,4,4,1,0,0,0,1,3,0
2,5419.631829,3.0,2,2,1,0,1,0,0,2,1
3,5419.631829,4.0,2,2,1,0,1,0,1,3,1
4,7420.000000,4.0,1,2,1,1,1,0,1,2,0
